# Solana Tax Reporter #
This notebook will produce a CSV file compatible with Koinly, which can be modified for other tax report formats.

The resulting file may possibly contain errors or omissions, so please check the file manually before using it. Trades involving more than 2 coins will split into parts. More complex transactions may need to be entered or edited manually.

In [ ]:
## Uncheck below if package is not yet already installed
#pip install base58

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime as dt
import base58
import base64
import struct

In [ ]:
#pubkey = 'ACCOUNT_PUBLIC_KEY'
pubkey = input('Enter account public key:')

In [ ]:
url = 'https://api.mainnet-beta.solana.com'

In [ ]:
def call(method, params):
    response = requests.post(url, headers = {'Content-Type': 'application/json'},
                            data = json.dumps({'jsonrpc': '2.0', 'id': 1, 'method': method, 'params': params}))
    return response

In [ ]:
response = call('getSignaturesForAddress', [pubkey])
sigs = [line['signature'] for line in response.json()['result']]
[str(len(sigs)) + " Transactions", sigs]

In [ ]:
df = pd.DataFrame(columns=['date', 'timestamp', 'preSOL', 'postSOL', 'fee', 'tokenTransfers', 'tokenBalances', 'txSig'])

In [ ]:
for sig in sigs:
    response = call('getTransaction', [sig])
    timestamp = response.json()['result']['blockTime']
    date = dt.strftime(dt.fromtimestamp(timestamp), '%Y-%m-%d %H:%M:%S')
    ix = response.json()['result']['transaction']['message']['accountKeys'].index(pubkey)
    preSOL = response.json()['result']['meta']['preBalances'][ix] / 10**9
    postSOL = response.json()['result']['meta']['postBalances'][ix] / 10**9
    reqSig = response.json()['result']['transaction']['message']['header']['numRequiredSignatures']
    if pubkey in response.json()['result']['transaction']['message']['accountKeys'][:reqSig]:
        fee = response.json()['result']['meta']['fee'] / 10**9
    else:
        fee = 0
    if len(response.json()['result']['meta']['preTokenBalances']) > 0:
        tokenTfs = []
        for i in range(len(response.json()['result']['meta']['preTokenBalances'])):
            if response.json()['result']['meta']['preTokenBalances'][i]['owner'] == pubkey:
                tokenTfs.append([response.json()['result']['meta']['preTokenBalances'][i]['mint'],
                                -(response.json()['result']['meta']['preTokenBalances'][i]['uiTokenAmount']['uiAmount'] or 0)])
        for i in range(len(response.json()['result']['meta']['postTokenBalances'])):
            if response.json()['result']['meta']['postTokenBalances'][i]['owner'] == pubkey:
                tokenTfs.append([response.json()['result']['meta']['postTokenBalances'][i]['mint'],
                                response.json()['result']['meta']['postTokenBalances'][i]['uiTokenAmount']['uiAmount'] or 0])
        df = df.append({'date': date, 'timestamp': timestamp, 'preSOL': preSOL, 'postSOL': postSOL,
                   'fee': fee, 'tokenTransfers': tokenTfs, 'tokenBalances': {}, 'txSig': sig}, ignore_index=True)
    else:
        df = df.append({'date': date, 'timestamp': timestamp, 'preSOL': preSOL,
                   'postSOL': postSOL, 'fee': fee, 'tokenBalances': {}, 'txSig': sig}, ignore_index=True)

In [ ]:
df

In [ ]:
# Token metadata search method adapted from
#   https://stackoverflow.com/questions/70481436/read-json-metadata-for-a-token-from-solscan/70481555#70481555

tokens = {}
headers = {'User-Agent':
           'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
for ix in df.index:
    if abs(df['postSOL'][ix] + df['fee'][ix] - df['preSOL'][ix]) > 1e-9:
        df.loc[ix, 'tokenBalances']['SOL'] = df['postSOL'][ix] + df['fee'][ix] - df['preSOL'][ix]
    if type(df['tokenTransfers'][ix]) == type([]):
        for bal in df['tokenTransfers'][ix]:
            if bal[0] not in tokens:
                params = {'address': bal[0]}
                response = requests.get('https://api.solscan.io/account', headers=headers, params=params)
                metadata = response.json()['data']['tokenInfo']
                if metadata['type'] == 'nft':
                    token = "_" + metadata['name']
                else:
                    token = metadata['symbol']
                tokens[bal[0]] = token
            if tokens[bal[0]] in df['tokenBalances'][ix]:
                df.loc[ix, 'tokenBalances'][tokens[bal[0]]] += bal[1]
            else:
                df.loc[ix, 'tokenBalances'][tokens[bal[0]]] = bal[1]

In [ ]:
tokens

In [ ]:
df.tokenBalances.to_dict()

In [ ]:
columns = ['Date', 'Sent Amount', 'Sent Currency', 'Received Amount', 'Received Currency',
           'Fee Amount', 'Fee Currency', 'Description', 'TxHash']
report = pd.DataFrame(columns=columns)

In [ ]:
for ix in df.index:
    toks = []; bals = []
    for k,v in df['tokenBalances'][ix].items():
        toks.append(k); bals.append(v)
    if len(df['tokenBalances'][ix]) == 1:
        if bals[0] > 0:
            recAmt = bals[0]
            if toks[0][0] == '_':
                recCur = 'NULL'; desc = toks[0][1:]
                report = report.append({'Date': df['date'][ix], 'Received Amount': recAmt, 'Received Currency': recCur,
                                        'Fee Amount': df['fee'][ix], 'Fee Currency': 'SOL', 'Description': desc,
                                        'TxHash': df['txSig'][ix]}, ignore_index=True)
            else:
                recCur = toks[0]
                report = report.append({'Date': df['date'][ix], 'Received Amount': recAmt, 'Received Currency': recCur,
                                        'Fee Amount': df['fee'][ix], 'Fee Currency': 'SOL', 'TxHash': df['txSig'][ix]},
                                      ignore_index=True)
        else:
            sentAmt = -bals[0]
            if toks[0][0] == '_':
                sentCur = 'NULL'; desc = toks[0][1:]
                report = report.append({'Date': df['date'][ix], 'Sent Amount':sentAmt, 'Sent Currency': sentCur,
                                        'Fee Amount': df['fee'][ix], 'Fee Currency': 'SOL', 'Description': desc,
                                        'TxHash': df['txSig'][ix]}, ignore_index=True)
            else:
                sentCur = toks[0]
                report = report.append({'Date': df['date'][ix], 'Sent Amount': sentAmt, 'Sent Currency': sentCur,
                                        'Fee Amount': df['fee'][ix], 'Fee Currency': 'SOL', 'TxHash': df['txSig'][ix]},
                                       ignore_index=True)
    elif len(df['tokenBalances'][ix]) == 2:
        if bals[0] > bals[1]:
            r = 0; s = 1
        else:
            r = 1; s = 0
        recAmt = bals[r]; sentAmt = -bals[s]; desc = 'Trade'
        if toks[r][0] == '_':
            recCur = 'NULL'; desc = toks[r][1:]
        else:
            recCur = toks[r]
        if toks[s][0] == '_':
            sentCur = 'NULL'; desc = toks[s][1:]
        else:
            sentCur = toks[s]
        report = report.append({'Date': df['date'][ix], 'Sent Amount': sentAmt, 'Sent Currency': sentCur,
                                'Received Amount': recAmt, 'Received Currency': recCur, 'Fee Amount': df['fee'][ix],
                                'Fee Currency': 'SOL', 'Description': desc, 'TxHash': df['txSig'][ix]}, ignore_index=True)
    elif len(df['tokenBalances'][ix]) > 2:
        for i in range(len(df['tokenBalances'][ix])):
            desc = "Multicoin trade: part " + str(i+1) + " of " + str(len(df['tokenBalances'][ix]))
            if i == 0:
                fee = df['fee'][ix]
            else:
                fee = 0
            if bals[i] > 0:
                recAmt = bals[i]
                if toks[i][0] == '_':
                    recCur = 'NULL'; desc = toks[i][1:]
                    report = report.append({'Date': df['date'][ix], 'Received Amount': recAmt, 'Received Currency': recCur,
                                            'Fee Amount': fee, 'Fee Currency': 'SOL', 'Description': desc,
                                            'TxHash': df['txSig'][ix]}, ignore_index=True)
                else:
                    recCur = toks[i]
                    report = report.append({'Date': df['date'][ix], 'Received Amount': recAmt, 'Received Currency': recCur,
                                            'Fee Amount': fee, 'Fee Currency': 'SOL', 'Description': desc,
                                            'TxHash': df['txSig'][ix]}, ignore_index=True)
            else:
                sentAmt = -bals[i]
                if toks[i][0] == '_':
                    sentCur = 'NULL'; desc = toks[i][1:]
                    report = report.append({'Date': df['date'][ix], 'Sent Amount':sentAmt, 'Sent Currency': sentCur,
                                            'Fee Amount': fee, 'Fee Currency': 'SOL', 'Description': desc,
                                            'TxHash': df['txSig'][ix]}, ignore_index=True)
                else:
                    sentCur = toks[i]
                    report = report.append({'Date': df['date'][ix], 'Sent Amount': sentAmt, 'Sent Currency': sentCur,
                                            'Fee Amount': fee, 'Fee Currency': 'SOL', 'Description': desc,
                                            'TxHash': df['txSig'][ix]}, ignore_index=True)

In [ ]:
report

In [ ]:
filename = "Solana_txs_" + pubkey + "_" + dt.strftime(dt.now(), "%Y%m%d_%H%M%S") + ".csv"

In [ ]:
report.to_csv(filename, index=False)